# 헝가리안 알고리즘에 필요한 가상 데이터 생성

In [1]:
from features_yj import Features

In [2]:
t = Features(types = "hungarian")
hung = t.run_all()

finish getting all lag data
(256740, 47) : df shape
finish getting brand power data
(256740, 48) : df shape
finish getting season_items data
(256740, 56) : df shape
finish getting click ratio data
(256740, 57) : df shape
finish getting click ratio data
(256740, 58) : df shape
finish getting click ratio data
(256740, 59) : df shape
finish getting weather data
(256740, 61) : df shape
finish getting add_device_click_ratio data
(256740, 80) : df shape
finish getting get_rolling_means data
(256740, 84) : df shape
finish getting get_mean_sales_origin data
(256740, 85) : df shape
finish getting get_lag_sales data
(256740, 89) : df shape
finish getting get_ts_pred data
(256740, 90) : df shape


# features 붙인 데이터에서 상품 리스트 만듦

In [555]:
hung_list = hung[['상품코드','상품명']].drop_duplicates()
hung_list['row_num'] = list(range(0,len(hung_list)))

,상품코드,상품명,row_num
0,200003,PN 메가티타늄 후라이팬+IH 인덕션 1구 풀세트,0
660,200004,PN 메가티타늄 후라이팬 기본세트,1
1320,200005,PN 프리미엄 IH 통3중 압력솥 베르투S+ 1구인덕션,2
1980,200006,W클라우드 남성 데일리 트레킹화 3종,3
2640,200007,W클라우드 여성 데일리 트레킹화 3종,4
...,...,...,...
253440,202485,단순생활 넥밴드 휴대용 선풍기,384
254100,202486,단순생활 넥밴드 휴대용 선풍기(1+1),385
254760,202506,추억의쥐치포110장,386
255420,202507,추억의쥐치포55장,387


# Import

In [131]:
# General imports
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import math
import random
import sys, gc, time
import os

# data
import datetime
import itertools
import json
import pickle

# visualize
# import seaborn as sns
# import matplotlib.pyplot as plt

from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA

# model
import lightgbm as lgb
from lightgbm import LGBMRegressor

# custom modules
# from engine.features_yj import Features
from preprocess import load_df_added, drop_useless, check_na, run_label_all, remove_outliers, run_stdscale

# 모델링 작업

In [11]:
lag_col1 = ['lag_scode_count','lag_mcode_price','lag_mcode_count','lag_bigcat_price','lag_bigcat_count',
            'lag_bigcat_price_day','lag_bigcat_count_day','lag_small_c_price','lag_small_c_count']

lag_col2 = ['rolling_mean_7', 'rolling_mean_14', 'rolling_mean_21', 'rolling_mean_28','mean_sales_origin',
            'lag_sales_wd_1', 'lag_sales_wd_2','lag_sales_wd_3','lag_sales_wd_4', 'lag_sales_wd_5', 
            'lag_sales_wk_1','lag_sales_wk_2', 'ts_pred']

cat_col = ['상품군','weekdays','show_id','small_c','middle_c','big_c','original_c',
                        'pay','months','hours_inweek','weekends','japp','parttime',
                        'min_start','primetime','prime_origin','prime_smallc',
                        'freq','bpower','steady','men','luxury',
                        'spring','summer','fall','winter','rain']

In [132]:
def drop_useless(df, keepshowid = True):
    """
    :objective: drop useless features for model. save 'show_id' just in case
    :return: pandas dataframe
    """
    #useless features
    xcol = ['방송일시','노출(분)', '마더코드', '상품명', 'exposed', 'ymd', 'volume',
            'years','days','hours','week_num','holidays', 'red', 'min_range','brand',
            'small_c_code','middle_c_code','big_c_code','sales_power']
    col = [x for x in df.columns if x in xcol]
    df = df.drop(columns = col)
    if keepshowid:
        df = df.copy()
    # else:
    #     df = df.drop(columns = ['show_id'])

    return df
    
def na_to_zeroes(df):
    """
    :objective: Change all na's to zero.(just for original lag!)
    :return: pandas dataframe
    """
    xcol = [x for x in df.columns if x in lag_col1+lag_col2]
    for col in xcol:
        df[col] = df[col].fillna(0)

    return df

def run_label_all(df):
    """
    :objective: Perform labelencoding for all categorical/object columns
    :return: pandas dataframe
    """
    lab_col = df.select_dtypes(include=['object','category']).columns.tolist()
    for col in lab_col:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].values)

    return df

## run preprocessing in a shot
## pca is optional and only applied to numeric features other than 'lag'
## NOTICE: removing outliers were run prior to dividing train/val
## if replace = True, new PCA will replace corresponding numerical columns
## if you want to simply add PCA columns to original data, set replace = False
def run_preprocess(df, pca = True, replace = True):
    """
    :objective: Run Feature deletion, NA imputation, label encoding, pca(optional)
    :return: pandas dataframe
    """
    df = drop_useless(df)
    df = na_to_zeroes(df)
    # df = remove_outliers(df)
    df = run_label_all(df)
    df1 = df.copy()
    return df1

In [17]:
def get_mape(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def get_rmse(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))
    return rmse

In [18]:
## Seeder
def seed_everything(seed=127):
    random.seed(seed)
    np.random.seed(seed)

## CV splits
def cv_split(df, month, printprop = False):
    split = int(df[df['months']==month].index.values.max())
    prop = str(split/df.shape[0])
    if printprop:
        print(f'Proportion of train set is {prop}')
        return split
    else:
        return split
        
## Divide into train/test
def divide_train_val(df_pp, month, drop):
    split = cv_split(df = df_pp, month = month)
    train_x = df_pp.iloc[:split].drop(['index','취급액']+drop, axis = 1)
    train_y = df_pp.iloc[:split,:].취급액
    val_x = df_pp.iloc[split:,:].drop(columns = ['index','취급액']+drop, axis = 1)
    val_y = df_pp.iloc[split:,:].취급액
    return train_x, train_y, val_x, val_y

In [133]:
hung_PP = run_preprocess(hung, pca = False, replace =False)
hung_PP = hung_PP.drop(['취급액','exposed_t'], axis=1)
hung_cols = hung_PP.columns.to_list()

In [214]:
hung_times = hung.iloc[:660]['방송일시'] # for output

In [139]:
df_full_lag = pd.read_pickle("../data/20/train_fin_light_ver.pkl")
df_full_lag = df_full_lag[hung_cols + ['취급액']]

In [140]:
df_full_lag = run_preprocess(df_full_lag, pca = False, replace =False)
df_full_lag.reset_index(inplace=True)
train_x, train_y, val_x, val_y = divide_train_val(df_full_lag, 8, drop = ['small_c','original_c','상품코드'])

In [145]:
params = {
    'feature_fraction': 1,
    'learning_rate': 0.001,
    'min_data_in_leaf': 135,
    'n_estimators': 3527,
    'num_iterations': 2940,
    'subsample': 1,
    'boosting_type': 'dart',
    'objective': 'regression',
    'metric': 'mape',
    'categorical_feature': [1,3,4,6,11,12,32]
}
gbm = LGBMRegressor(**params)


def run_lgbm(train_x, train_y, val_x, val_y):
    seed_everything(seed=127)
    estimator = gbm.fit(train_x,train_y,
                      eval_set=[(val_x, val_y)],
                      verbose = 100,
                      eval_metric = 'mape',
                      early_stopping_rounds = 100
                      )
    lgbm_preds = gbm.predict(val_x, num_iteration= estimator.best_iteration_)
    lgbm_preds[lgbm_preds < 0] = 0

    # plot
    x = range(0,val_y.shape[0])
    plt.figure(figsize=(50,10))
    plt.plot(x,val_y,label='true')
    plt.plot(x,lgbm_preds, label='predicted')
    plt.legend()

    plt.show()

    # show scores
    print(f'MAPE of best iter is {get_mape(val_y,lgbm_preds)}')
    print(f'RMSE of best iter is {get_rmse(val_y,lgbm_preds)}')

    # save model
    data_type = 'all_lag'
    model_name = '../data/'+'lgbm_opt_mape_lr001_'+data_type+'.bin'
    pickle.dump(estimator, open(model_name, 'wb'))


In [146]:
run_lgbm(train_x, train_y, val_x, val_y)

[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] min_data_in_leaf is set=135, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=135
[100]	valid_0's mape: 1.01635
[200]	valid_0's mape: 0.777444
[300]	valid_0's mape: 0.738704
[400]	valid_0's mape: 0.717159
[500]	valid_0's mape: 0.698214
[600]	valid_0's mape: 0.692784
[700]	valid_0's mape: 0.679594
[800]	valid_0's mape: 0.671166
[900]	valid_0's mape: 0.662533
[1000]	valid_0's mape: 0.656354
[1100]	valid_0's mape: 0.647965
[1200]	valid_0's mape: 0.642991
[1300]	valid_0's mape: 0.63868
[1400]	valid_0's mape: 0.631665
[1500]	valid_0's mape: 0.629516
[1600]	valid_0's mape: 0.625772
[1700]	valid_0's mape: 0.618158
[1800]	valid_0's mape: 0.61325
[1900]	valid_0's mape: 0.610746
[2000]	valid_0's mape: 0.607608
[2100]	valid_0's mape: 0.604878
[2200]	valid_0's mape: 0.603197
[2300]	valid_0's mape: 0.599753
[2400]	valid_0's mape: 0.59958
[

NameError: name 'plt' is not defined

# 경민이 bin 파일로 돌림. 시발?

In [148]:
# 이게 그냥 되는데, 애초에 왜 위에걸 했지..?
lgbm_model_path = "../data/lgbm_finalmodel_.bin"
estimator = pickle.load(open(lgbm_model_path, 'rb'))
lgbm_preds_opt = estimator.predict(hung_PP.drop(columns=['small_c']))

# 전처리된 가상 데이터에 예측값 붙이고 다듬기

In [217]:
hung_PP['pred'] = lgbm_preds_opt
hung_PP['방송일시'] = hung.방송일시
hung_PP['ymd'] = [d.date() for d in hung_PP["방송일시"]]

# 헝가리안 알고리즘 적용

In [433]:
from munkres import Munkres, print_matrix, make_cost_matrix
import datetime

prod_list = pd.read_excel('../data/01/2020sales_opt_temp_list.xlsx')

Timestamp('2020-06-02 01:00:00', freq='D')

In [635]:
h1_output = []
h2_output = []

for i in pd.date_range(start=datetime.datetime(2020, 6, 2, 1),end=datetime.datetime(2020, 7, 1, 1)):
    
    if i == datetime.datetime(2020,6,2,1) :
        jun_day = hung_PP.loc[ hung_PP.방송일시 <= i,:]
    else :
        i2 = i - datetime.timedelta(days=1)
        jun_day = hung_PP.loc[ (hung_PP.방송일시 > i2) & (hung_PP.방송일시 <= i),:]

    ind = np.random.multinomial(n=22, pvals = [0.025, 0.115, 0.009, 0.217, 0.12, 0.121, 0.093, 0.107, 0.069, 0.113, 0.009])
    rmv_cat = np.where(np.isin(ind,0))[0].tolist()
    jun_day_selected = jun_day.loc[~jun_day.상품군.isin(rmv_cat),:]
    jun_day_grp = jun_day_selected.groupby(['방송일시','상품군']).pred.mean().to_frame()
    jun_day_grp.reset_index(inplace = True)
    jun_day_pvt = jun_day_grp.pivot(index = '방송일시', columns='상품군')
    jun_day_hung = jun_day_pvt[np.repeat(jun_day_pvt.columns.values, ind[~np.isin(ind,0)].tolist())]

    matrix = np.array(jun_day_hung)
    cost_matrix = []
    for row in matrix:
        cost_row = []
        for col in row:
            cost_row += [sys.maxsize - col]
        cost_matrix += [cost_row]

    m = Munkres()

    indexes = m.compute(cost_matrix)
    total = 0
    for row, column in indexes:
        value = matrix[row][column]
        total += value


    jun_day_hung.reset_index(inplace = True)
    cl = np.repeat(jun_day_pvt.columns.values, ind[~np.isin(ind,0)].tolist())
    jun_day_fin = jun_day_hung.iloc[:,0].to_frame()
    jun_day_fin['상품군'] = 0
    for id in indexes:
        jun_day_fin.iloc[id[0],1] = cat[cl[id[1]][1]]

    h1_output.append(jun_day_fin)
    
    

    cat_temp = np.where(~np.isin(ind,0))[0].tolist()
    if i < datetime.datetime(2020, 6, 29, 1):
        jun_week = hung_PP.loc[(hung_PP.방송일시.dt.week > i.isocalendar()[1]-1) & 
                                (hung_PP.방송일시.dt.week <= i.isocalendar()[1]),:]
    else:
        jun_week = hung_PP.loc[(hung_PP.방송일시.dt.week >= i.isocalendar()[1]-1),:]


    for j in cat_temp:

        runprod = jun_week.loc[jun_week.상품군 == j,:].상품코드.unique().tolist()
        runtime = jun_day_fin.loc[jun_day_fin.상품군 == cat[j],'방송일시']

        run_schd = jun_day.loc[jun_day.상품코드.isin(runprod) & jun_day.방송일시.isin(runtime),].groupby(['상품코드','방송일시']).pred.mean().to_frame()
        run_schd.reset_index(inplace = True)
        day_cat_hung = run_schd.pivot(index = '방송일시', columns='상품코드')

        matrix = np.array(day_cat_hung)
        cost_matrix = []
        for row in matrix:
            cost_row = []
            for col in row:
                cost_row += [sys.maxsize - col]
            cost_matrix += [cost_row]

        m = Munkres()
        indexes = m.compute(cost_matrix)
        # print_matrix(matrix, msg='Highest profit through this matrix:')
        total = 0
        for row, column in indexes:
            value = matrix[row][column]
            total += value
        #     print(f'({row}, {column}) -> {value}')

        print(f'total profit={total}')

        code = []
        ct = []
        for id2 in indexes:
            code.append((run_schd.상품코드.unique().tolist()[id2[1]]))

        prod_fin = pd.DataFrame({'방송일시':run_schd.방송일시.unique(),
                                 '상품코드':code})

        h2_output.append(prod_fin)

    h2_output_fin = pd.concat(h2_output)

h1_output = pd.concat(h1_output)

h2_output_fin = h2_output_fin.sort_values('방송일시')
h2_output_name = pd.merge(h2_output_fin, hung_list, left_on='상품코드', right_on='row_num' ,how = 'left')
h2_output_name = h2_output_name.drop(['상품코드_x'], axis=1)
h2_output_name = h2_output_name.rename(columns = {'상품코드_y' : '상품코드'})
h2_output_final = pd.merge(h2_output_name, prod_list, on =['상품코드','상품명'], how='left').drop('row_num',1)

total profit=16507169.68598935
total profit=9867229.08620473
total profit=90678909.79598497
total profit=8266075.7642571
total profit=31784997.53310795
total profit=63569995.0662159
total profit=111396178.98364033
total profit=8036993.145223051
total profit=53088290.86423324
total profit=33073326.918863464
total profit=160848970.76308823
total profit=48680124.109406404
total profit=63569995.0662159
total profit=37513119.43169163
total profit=8036993.145223051
total profit=17758784.974844165
total profit=8932979.864121003
total profit=49316881.75068217
total profit=160848970.76308823
total profit=8266075.7642571
total profit=63633205.3947204
total profit=63633205.3947204
total profit=38123254.924107395
total profit=8185155.548113044
total profit=8815207.422680294
total profit=6702368.822454655
total profit=39454468.58456281
total profit=160848970.76308823
total profit=24569144.67373725
total profit=15908301.3486801
total profit=15892498.766553976
total profit=37487842.22936549
total pro

# 최적 상품군 결과, 최적 상품명 결과 내보내기

In [634]:
import os

h1_output['n'] = 1
h1_output.pivot_table(index = '방송일시', columns = '상품군', fill_value = 0 ).to_excel('../data/h1_output.xlsx')

h2_output_final.to_excel('../data/h2_output.xlsx')

# Multinomial prob 구하기

In [ ]:
tmp = pd.read_pickle("../data/20/test_v2.pkl")
# tmp.loc[(tmp.months == 6) & (tmp.days == 4),]
tmp2 = tmp.loc[tmp.months==6,].groupby(['days','상품군']).show_id.nunique().to_frame()
tmp2.reset_index(inplace=True)
tmp3 = tmp2.pivot_table(index='days', columns='상품군',aggfunc=sum, margins=True, 
                       dropna=True, fill_value=0)

tmp4 = tmp3.div( tmp3.iloc[:,-1], axis=0 )
tmp4.mean(axis=0)

# 원래 편성안의 상품에서 최적 편성안의 상품이 차지하는 비율

In [650]:
np.isin(tmp.상품명.unique(),h2_output_final.상품명.unique()).sum() / len(tmp.상품명.unique())

0.15472779369627507

In [361]:
cl = np.repeat(jun_day1_pvt.columns.values, ind[~np.isin(ind,0)].tolist())
jun_day1_fin = jun_day1_hung.iloc[:,0].to_frame()
jun_day1_fin['상품군'] = 0
for i in indexes:
    jun_day1_fin.iloc[i[0],1] = cat[cl[i[1]][1]]

In [362]:
jun_day1_fin

,방송일시,상품군
,,
0,2020-06-01 02:00:00,잡화
1,2020-06-01 04:00:00,생활용품
2,2020-06-01 06:00:00,잡화
3,2020-06-01 07:00:00,의류
4,2020-06-01 08:00:00,의류
5,2020-06-01 09:00:00,의류
6,2020-06-01 10:00:00,의류
7,2020-06-01 11:00:00,속옷
8,2020-06-01 12:00:00,속옷


In [382]:
cat_temp = np.where(~np.isin(ind,0))[0].tolist()
jun_week1 = hung_PP.loc[hung_PP.방송일시 <= datetime.datetime(2020, 6, 8, 1),:]
runprod = jun_week1.loc[jun_week1.상품군 == cat_temp[0],:].상품코드.unique().tolist()
runtime = jun_day1_fin.loc[jun_day1_fin.상품군 == cat[cat_temp[0]],'방송일시']

In [385]:
runtime

9    2020-06-01 13:00:00
10   2020-06-01 14:00:00
14   2020-06-01 18:00:00
15   2020-06-01 19:00:00
16   2020-06-01 20:00:00
17   2020-06-01 21:00:00
18   2020-06-01 22:00:00
19   2020-06-01 23:00:00
Name: 방송일시, dtype: datetime64[ns]

In [413]:
run_schd = jun_day1.loc[jun_day1.상품코드.isin(runprod) & jun_day1.방송일시.isin(runtime),].groupby(['상품코드','방송일시']).pred.mean().to_frame()
run_schd.reset_index(inplace = True)
day1_cat1_hung = run_schd.pivot(index = '방송일시', columns='상품코드')
day1_cat1_hung

pred                                            \
상품코드                          7             8             9             10    
방송일시                                                                          
2020-06-01 13:00:00  3.020859e+07  3.020859e+07  2.989595e+07  3.020859e+07   
2020-06-01 14:00:00  3.020859e+07  3.020859e+07  2.989595e+07  3.020859e+07   
2020-06-01 18:00:00  3.020859e+07  3.020859e+07  2.989595e+07  3.020859e+07   
2020-06-01 19:00:00  3.020859e+07  3.020859e+07  2.989595e+07  3.020859e+07   
2020-06-01 20:00:00  3.020859e+07  3.020859e+07  2.989595e+07  3.020859e+07   
2020-06-01 21:00:00  3.020859e+07  3.020859e+07  2.989595e+07  3.020859e+07   
2020-06-01 22:00:00  3.020859e+07  3.020859e+07  2.989595e+07  3.020859e+07   
2020-06-01 23:00:00  3.020859e+07  3.020859e+07  2.989595e+07  3.020859e+07   

                                                                             \
상품코드                          11            12            13            16    
방송일시                                                                          
2020-06-01 13:00:00  2.989595e+07  2.989595e+07  3.020859e+07  3.020859e+07   
2020-06-01 14:00:00  2.989595e+07  2.989595e+07  3.020859e+07  3.020859e+07   
2020-06-01 18:00:00  2.989595e+07  2.989595e+07  3.020859e+07  3.020859e+07   
2020-06-01 19:00:00  2.989595e+07  2.989595e+07  3.020859e+07  3.020859e+07   
2020-06-01 20:00:00  2.989595e+07  2.989595e+07  3.020859e+07  3.020859e+07   
2020-06-01 21:00:00  2.989595e+07  2.989595e+07  3.020859e+07  3.020859e+07   
2020-06-01 22:00:00  2.989595e+07  2.989595e+07  3.020859e+07  3.020859e+07   
2020-06-01 23:00:00  2.989595e+07  2.989595e+07  3.020859e+07  3.020859e+07   

                                                 ...                \
상품코드                          32            77   ...           353   
방송일시                                             ...                 
2020-06-01 13:00:00  3.020859e+07  3.020859e+07  ...  2.989595e+07   
2020-06-01 14:00:00  3.020859e+07  3.020859e+07  ...  2.989595e+07   
2020-06-01 18:00:00  3.020859e+07  3.020859e+07  ...  2.989595e+07   
2020-06-01 19:00:00  3.020859e+07  3.020859e+07  ...  2.989595e+07   
2020-06-01 20:00:00  3.020859e+07  3.020859e+07  ...  2.989595e+07   
2020-06-01 21:00:00  3.020859e+07  3.020859e+07  ...  2.989595e+07   
2020-06-01 22:00:00  3.020859e+07  3.020859e+07  ...  2.989595e+07   
2020-06-01 23:00:00  3.020859e+07  3.020859e+07  ...  2.989595e+07   

                                                                             \
상품코드                          360           367           369           379   
방송일시                                                                          
2020-06-01 13:00:00  3.020859e+07  3.020859e+07  3.020859e+07  3.020859e+07   
2020-06-01 14:00:00  3.020859e+07  3.020859e+07  3.020859e+07  3.020859e+07   
2020-06-01 18:00:00  3.020859e+07  3.020859e+07  3.020859e+07  3.020859e+07   
2020-06-01 19:00:00  3.020859e+07  3.020859e+07  3.020859e+07  3.020859e+07   
2020-06-01 20:00:00  3.020859e+07  3.020859e+07  3.020859e+07  3.020859e+07   
2020-06-01 21:00:00  3.020859e+07  3.020859e+07  3.020859e+07  3.020859e+07   
2020-06-01 22:00:00  3.020859e+07  3.020859e+07  3.020859e+07  3.020859e+07   
2020-06-01 23:00:00  3.020859e+07  3.020859e+07  3.020859e+07  3.020859e+07   

                                                                             \
상품코드                          380           383           386           387   
방송일시                                                                          
2020-06-01 13:00:00  3.020859e+07  3.020859e+07  3.020859e+07  3.020859e+07   
2020-06-01 14:00:00  3.020859e+07  3.020859e+07  3.020859e+07  3.020859e+07   
2020-06-01 18:00:00  3.020859e+07  3.020859e+07  3.020859e+07  3.020859e+07   
2020-06-01 19:00:00  3.020859e+07  3.020859e+07  3.020859e+07  3.020859e+07   
2020-06-01 20:00:00  3.020859e+07  3.020859e+07  3.020859e+07  3.020859e+07   
2020-06-01 21:00

In [400]:
matrix = np.array(day1_cat1_hung)
cost_matrix = []
for row in matrix:
    cost_row = []
    for col in row:
        cost_row += [sys.maxsize - col]
    cost_matrix += [cost_row]

In [401]:
m = Munkres()
indexes = m.compute(cost_matrix)
# print_matrix(matrix, msg='Highest profit through this matrix:')
total = 0
for row, column in indexes:
    value = matrix[row][column]
    total += value
#     print(f'({row}, {column}) -> {value}')

print(f'total profit={total}')

total profit=241721852.8872963


In [403]:
indexes

[(0, 22), (1, 21), (2, 66), (3, 65), (4, 64), (5, 63), (6, 62), (7, 61)]

In [429]:
code = []
for i in indexes:
    code.append((run_schd.상품코드.unique().tolist()[i[1]]))
    
prod_fin = pd.DataFrame({'방송일시':run_schd.방송일시.unique(),
             '상품코드':code})

,방송일시,상품코드
0,2020-06-01 13:00:00,180
1,2020-06-01 14:00:00,179
2,2020-06-01 18:00:00,387
3,2020-06-01 19:00:00,386
4,2020-06-01 20:00:00,383
5,2020-06-01 21:00:00,380
6,2020-06-01 22:00:00,379
7,2020-06-01 23:00:00,369


In [155]:
hung_mat = lgbm_preds_opt.reshape((660,389)) #rows: items  cols: time

In [157]:
hung_mat.shape

(660, 389)

In [165]:
hung_mat

array([[ 8642573.05166171,  8691328.42712283,  8879392.48742208, ...,
         8879392.48742208,  8879392.48742208,  8879392.48742208],
       [ 8879392.48742208,  8879392.48742208,  8879392.48742208, ...,
         8879392.48742208,  8879392.48742208,  8879392.48742208],
       [ 8879392.48742208,  8879392.48742208,  8879392.48742208, ...,
         8879392.48742208,  8815207.42268029,  8642573.05166171],
       ...,
       [22644622.48465185, 23158420.36476086, 24028030.69097909, ...,
        32934668.05707845, 32934668.05707845, 32934668.05707845],
       [32934668.05707845, 32934668.05707845, 32934668.05707845, ...,
        32102862.17008805, 32102862.17008805, 32102862.17008805],
       [32102862.17008805, 32102862.17008805, 32102862.17008805, ...,
        32934668.05707845, 31162375.44230326, 22910888.0374084 ]])

         상품군 
show_id  가구      0.025323
         가전      0.114637
         건강기능    0.009194
         농수축     0.217572
         생활용품    0.119569
         속옷      0.121465
         의류      0.093029
         이미용     0.107931
         잡화      0.069061
         주방      0.113025
         침구      0.009194
         All     1.000000
dtype: float64

In [251]:
ind = np.random.multinomial(n=22, pvals = [0.025, 0.115, 0.009, 0.217, 0.12, 0.121, 0.093, 0.107, 0.069, 0.113, 0.009])

array([ 8642573.05166171,  8691328.42712283,  8879392.48742208, ...,
       32934668.05707845, 31162375.44230326, 22910888.0374084 ])

In [ ]:
train_x['상품군'] = pd.factorize(train_x['상품군'])[0]
train_x['big_c'] = pd.factorize(train_x['big_c'])[0]
train_x['middle_c'] = pd.factorize(train_x['middle_c'])[0]
train_x['weekdays'] = pd.factorize(train_x['weekdays'])[0]
train_x['parttime'] = pd.factorize(train_x['parttime'])[0]
train_x['show_id'] = pd.factorize(train_x['show_id'])[0]
train_x['pay'] = pd.factorize(train_x['pay'])[0]

In [2]:
import pandas as pd

hung_wd_pred = pd.read_pickle('../data/20/hung_wd_lag_y.pkl')
hung_wk_pred = pd.read_pickle('../data/20/hung_wk_lag_y.pkl')

In [16]:
from itertools import chain, repeat

hung_wd_pred['row_num'] = list(chain.from_iterable(repeat(list(range(0,1377)), 389)))

In [17]:
profit_matrix = hung_wd_pred[['상품코드','predicted','row_num']].pivot(index = '상품코드', columns = 'row_num')

In [21]:
hung_wd_pred.columns

Index(['상품코드', '상품군', '판매단가', 'big_c', 'middle_c', 'small_c', 'original_c',
       'months', 'weekdays', 'hours_inweek', 'weekends', 'min_start', 'japp',
       'parttime', 'show_id', 'primetime', 'prime_smallc', 'freq', 'bpower',
       'steady', 'men', 'luxury', 'pay', 'spring', 'summer', 'fall', 'winter',
       'small_click_r', 'mid_click_r', 'big_click_r', 'rain', 'temp_diff_s',
       'age30_small', 'age40_small', 'age50_small', 'age60above_small',
       'age30_middle', 'age40_middle', 'age50_middle', 'age60above_middle',
       'age30_big', 'age40_big', 'age50_big', 'age60above_big', 'pc_small',
       'mobile_small', 'pc_middle', 'mobile_middle', 'pc_big', 'mobile_big',
       'rolling_mean_7', 'rolling_mean_14', 'rolling_mean_21',
       'rolling_mean_28', 'mean_sales_origin', 'lag_sales_wd_1',
       'lag_sales_wd_2', 'lag_sales_wd_3', 'lag_sales_wd_4', 'lag_sales_wd_5',
       'ts_pred', 'predicted', 'row_num'],
      dtype='object')

In [ ]:
hung_wd_pred[['상품코드','상품군','predicted','row_num']]

In [63]:
from hungarian import Hungarian

hungarian = Hungarian(profit_matrix, is_profit_matrix=True)
hungarian.calculate()

KeyboardInterrupt: 

In [ ]:
profit_matrix = [[5, 9, 1, 2],
          [10, 3, 2, 5],
          [8, 7, 4, 9]]
profit_matrix

In [ ]:
cost_matrix = make_cost_matrix(profit_matrix)

m = Munkres()
indexes = m.compute(cost_matrix)
print_matrix(matrix, msg='Highest profits through this matrix:')
total = 0
for row, column in indexes:
    value = matrix[row][column]
    total += value
    print(f'(${row}, ${column}) -> ${total}')
print(f'total profit=${total}')

In [60]:
hung_wd_pred.columns

Index(['상품코드', '상품군', '판매단가', 'big_c', 'middle_c', 'small_c', 'original_c',
       'months', 'weekdays', 'hours_inweek', 'weekends', 'min_start', 'japp',
       'parttime', 'show_id', 'primetime', 'prime_smallc', 'freq', 'bpower',
       'steady', 'men', 'luxury', 'pay', 'spring', 'summer', 'fall', 'winter',
       'small_click_r', 'mid_click_r', 'big_click_r', 'rain', 'temp_diff_s',
       'age30_small', 'age40_small', 'age50_small', 'age60above_small',
       'age30_middle', 'age40_middle', 'age50_middle', 'age60above_middle',
       'age30_big', 'age40_big', 'age50_big', 'age60above_big', 'pc_small',
       'mobile_small', 'pc_middle', 'mobile_middle', 'pc_big', 'mobile_big',
       'rolling_mean_7', 'rolling_mean_14', 'rolling_mean_21',
       'rolling_mean_28', 'mean_sales_origin', 'lag_sales_wd_1',
       'lag_sales_wd_2', 'lag_sales_wd_3', 'lag_sales_wd_4', 'lag_sales_wd_5',
       'ts_pred', 'predicted', 'row_num'],
      dtype='object')

In [35]:
hung_pred.groupby('상품코드').count()

,상품군,판매단가,big_c,middle_c,small_c,original_c,months,weekdays,hours_inweek,weekends,...,rolling_mean_21,rolling_mean_28,mean_sales_origin,lag_sales_wd_1,lag_sales_wd_2,lag_sales_wd_3,lag_sales_wd_4,lag_sales_wd_5,ts_pred,predicted
상품코드,,,,,,,,,,,,,,,,,,,,,
0,1377,1377,1377,1377,1377,1377,1377,1377,1377,1377,...,1377,1377,1377,1377,1377,1377,1377,1377,1377,1377
1,1377,1377,1377,1377,1377,1377,1377,1377,1377,1377,...,1377,1377,1377,1377,1377,1377,1377,1377,1377,1377
2,1377,1377,1377,1377,1377,1377,1377,1377,1377,1377,...,1377,1377,1377,1377,1377,1377,1377,1377,1377,1377
3,1377,1377,1377,1377,1377,1377,1377,1377,1377,1377,...,1377,1377,1377,1377,1377,1377,1377,1377,1377,1377
4,1377,1377,1377,1377,1377,1377,1377,1377,1377,1377,...,1377,1377,1377,1377,1377,1377,1377,1377,1377,1377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
384,1377,1377,1377,1377,1377,1377,1377,1377,1377,1377,...,1377,1377,1377,1377,1377,1377,1377,1377,1377,1377
385,1377,1377,1377,1377,1377,1377,1377,1377,1377,1377,...,1377,1377,1377,1377,1377,1377,1377,1377,1377,1377
386,1377,1377,1377,1377,1377,1377,1377,1377,1377,1377,...,1377,1377,1377,1377,1377,1377,1377,1377,1377,1377


In [1]:
matrix = [[5, 9, 1],
          [10, 3, 2],
          [8, 7, 4]]

In [2]:
def permute(a, results):
    if len(a) == 1:
        results.insert(len(results), a)
    else:
        for i in range(0, len(a)):
            element = a[i]
            a_copy = [a[j] for j in range(0, len(a)) if j != i]
            subresults = []
            permute(a_copy, subresults)
            for subresult in subresults:
                result = [element] + subresult
                results.insert(len(results), result)
                
results = []
permute(range(len(matrix)), results) # [0, 1, 2] for a 3x3 matrix

In [4]:
results

[[0, 1, 2], [0, 2, 1], [1, 0, 2], [1, 2, 0], [2, 0, 1], [2, 1, 0]]

In [1]:
from munkres import Munkres

In [2]:
m = Munkres()

In [4]:
from munkres import Munkres, print_matrix

matrix = [[5, 9, 1],
          [10, 3, 2],
          [8, 7, 4]]
m = Munkres()
indexes = m.compute(matrix)
print_matrix(matrix, msg='Lowest cost through this matrix:')
total = 0
for row, column in indexes:
    value = matrix[row][column]
    total += value
    print(f'({row}, {column}) -> {value}')
print(f'total cost: {total}')

Lowest cost through this matrix:
[ 5,  9,  1]
[10,  3,  2]
[ 8,  7,  4]
(0, 2) -> 1
(1, 1) -> 3
(2, 0) -> 8
total cost: 12


In [8]:
from munkres import Munkres, print_matrix, make_cost_matrix

matrix = [[5, 9, 1],
          [10, 3, 2],
          [8, 7, 4]]
cost_matrix = make_cost_matrix(matrix)

In [10]:
cost_matrix = make_cost_matrix(matrix)

m = Munkres()
indexes = m.compute(cost_matrix)
print_matrix(matrix, msg='Highest profits through this matrix:')
total = 0
for row, column in indexes:
    value = matrix[row][column]
    total += value
    print(f'(${row}, ${column}) -> ${total}')
print(f'total profit=${total}')

Highest profits through this matrix:
[ 5,  9,  1]
[10,  3,  2]
[ 8,  7,  4]
($0, $1) -> $9
($1, $0) -> $19
($2, $2) -> $23
total profit=$23


[[5, 1, 9], [0, 7, 8], [2, 3, 6]]

In [28]:
profit_matrix = [[5, 9, 1, 2],
          [10, 3, 2, 5],
          [8, 7, 4, 9]]
profit_matrix

[[5, 9, 1, 2], [10, 3, 2, 5], [8, 7, 4, 9]]

In [31]:
hungarian = Hungarian(profit_matrix, is_profit_matrix=True)
hungarian.calculate()
hungarian.get_results()

[(0, 1), (1, 0), (2, 3)]

In [12]:
from hungarian import Hungarian

In [18]:
# hungarian = Hungarian(cost_matrix)
# hungarian.calculate()

hungarian = Hungarian()
hungarian.calculate(cost_matrix)

[(0, 1), (1, 0), (2, 2)]

In [23]:
hungarian.get_total_potential()

7

In [ ]:
hungarian = Hungarian(profitMatrix, isProfitMatrix=True)
# costMatrix = Hungarian.makeCostMatrix(profitMatrix)

In [ ]:




paddedMatrix = Hungarian.padMatrix(costMatrix)



In [1]:
from features_yj import Features

In [2]:
rtn = Features(types = 'hungarian')

In [11]:
# rtn.get_time()
# rtn.get_weekday()
# rtn.get_hours_inweek()
# rtn.get_holidays()
# rtn.get_red_days()
# rtn.get_weekends()
# rtn.get_min_start()
# rtn.filter_jappingt()
# rtn.fill_exposed_na()
# rtn.round_exposed()

## rtn.get_ymd()
## rtn.timeslot()
## rtn.get_show_id()
## rtn.get_min_range()
## rtn.add_showid_minran_to_train()

# rtn.get_primetime()
# rtn.check_smallc_primet()

## rtn.get_sales_power()
## rtn.freq_items()
## rtn.check_steady_sellers()
## rtn.check_brand_power()

# rtn.check_men_items()
# rtn.check_luxury_items()
# rtn.check_pay()

## rtn.get_dup_times()
## rtn.get_dup_times_smallc()

# rtn.get_lag_scode_price()

## rtn.get_lag_scode_count()
# rtn.get_lag_mcode_price()
## rtn.get_lag_mcode_count()
# rtn.get_lag_bigcat_price()
## rtn.get_lag_bigcat_count()
# rtn.get_lag_bigcat_price_day()
## rtn.get_lag_bigcat_count_day()
# rtn.get_lag_small_c_price()
## rtn.get_lag_small_c_count()
# rtn.get_lag_all_price_show()
# rtn.get_lag_all_price_day()

# rtn.get_lag_sales()
# rtn.get_rolling_means()
# rtn.get_mean_sales_origin()


In [13]:
rtn.get_df().parttime

0            1
1            1
2            1
3            1
4            1
          ... 
746842    1665
746843    1665
746844    1665
746845    1665
746846    1665
Name: parttime, Length: 746847, dtype: object

In [67]:
rtn.get_df().dup_times.value_counts()

2    746847
Name: dup_times, dtype: int64

In [77]:
rtn.get_df()

,상품코드,방송일시,마더코드,상품명,상품군,판매단가,취급액,big_c_code,big_c,middle_c_code,...,dup_times,lag_scode_price,lag_scode_count,lag_mcode_price,lag_mcode_count,lag_bigcat_price,lag_bigcat_price_day,lag_small_c_price,lag_all_price_show,lag_all_price_day
0,200003,2020-06-01 06:20:00,100003,PN 메가티타늄 후라이팬+IH 인덕션 1구 풀세트,주방,98000,NaN,50000008.0,생활/건강,50000061.0,...,2,NaN,0,NaN,0,NaN,0.000000e+00,NaN,0.000000,0.000000
1791,200004,2020-06-01 06:20:00,100003,PN 메가티타늄 후라이팬 기본세트,주방,68000,NaN,50000008.0,생활/건강,50000061.0,...,2,NaN,0,NaN,0,NaN,0.000000e+00,NaN,0.000000,0.000000
3582,200005,2020-06-01 06:20:00,100004,PN 프리미엄 IH 통3중 압력솥 베르투S+ 1구인덕션,주방,199000,NaN,50000008.0,생활/건강,50000061.0,...,2,NaN,0,NaN,0,NaN,0.000000e+00,NaN,0.000000,0.000000
5373,200006,2020-06-01 06:20:00,100005,W클라우드 남성 데일리 트레킹화 3종,잡화,49800,NaN,50000007.0,스포츠/레저,50000027.0,...,2,NaN,0,NaN,0,NaN,0.000000e+00,NaN,0.000000,0.000000
7164,200007,2020-06-01 06:20:00,100005,W클라우드 여성 데일리 트레킹화 3종,잡화,49800,NaN,50000007.0,스포츠/레저,50000027.0,...,2,NaN,0,NaN,0,NaN,0.000000e+00,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739682,202486,2020-07-01 01:40:00,100842,단순생활 넥밴드 휴대용 선풍기(1+1),생활용품,59900,NaN,50000003.0,디지털/가전,50000212.0,...,2,42900.0,0,5.140000e+04,0,1.599983e+05,NaN,5.140000e+04,396454.556355,396454.556355
741473,202506,2020-07-01 01:40:00,100847,추억의쥐치포110장,농수축,32900,NaN,50000006.0,식품,50000159.0,...,2,849000.0,0,1.048167e+06,0,1.390178e+06,NaN,1.266647e+06,396454.556355,396454.556355
743264,202507,2020-07-01 01:40:00,100847,추억의쥐치포55장,농수축,32900,NaN,50000006.0,식품,50000159.0,...,2,39900.0,0,3.990000e+04,0,7.293333e+04,NaN,7.243571e+04,396454.556355,396454.556355
745055,202508,2020-07-01 01:40:00,100848,TCOM_현대해상 (무)하이콜운전자상해보험,무형,0,NaN,NaN,NaN,NaN,...,2,1100000.0,0,1.620000e+06,0,1.390178e+06,1.390178e+06,1.738333e+06,396454.556355,396454.556355


In [ ]:

rtn.get_df().상품코드.astype(int).astype(str).str.zfill(6)


In [47]:
import pandas as pd

df = pd.read_excel("../data/00/202006schedule.xlsx", skiprows=1)

df['exposed'] = df['노출(분)']

testpkl = pd.read_pickle('../data/20/test_v2.pkl')

In [53]:
testpkl.columns

Index(['방송일시', '노출(분)', '마더코드', '상품코드', '상품명', '상품군', '판매단가', '취급액', 'exposed',
       'ymd', 'years', 'months', 'days', 'hours', 'week_num', 'weekdays',
       'hours_inweek', 'holidays', 'red', 'weekends', 'min_start', 'japp',
       'parttime', 'min_range', 'show_id', 'small_c', 'small_c_code',
       'middle_c', 'middle_c_code', 'big_c', 'big_c_code', 'original_c',
       'primetime', 'prime_smallc', 'freq', 'dup_times', 'dup_times_smallc',
       'lag_scode_count', 'lag_mcode_price', 'lag_mcode_count',
       'lag_bigcat_price', 'lag_bigcat_count', 'lag_bigcat_price_day',
       'lag_bigcat_count_day', 'lag_small_c_price', 'lag_small_c_count',
       'lag_all_price_show', 'lag_all_price_day', 'bpower', 'steady', 'men',
       'luxury', 'pay', 'vratings', 'spring', 'summer', 'fall', 'winter',
       'small_click_r', 'mid_click_r', 'big_click_r', 'rain', 'temp_diff_s',
       'exposed_t', 'age30_small', 'age40_small', 'age50_small',
       'age60above_small', 'age30_middle', 'age40_

In [55]:
testpkl.freq.sum()

0

In [18]:
df.마더코드

0       100650
1       100650
2       100650
3       100445
4       100445
         ...  
2886    100660
2887    100660
2888    100660
2889    100261
2890    100261
Name: 마더코드, Length: 2891, dtype: int64

In [19]:
df.마더코드.astype(int).astype(str).str.zfill(6)

0       100650
1       100650
2       100650
3       100445
4       100445
         ...  
2886    100660
2887    100660
2888    100660
2889    100261
2890    100261
Name: 마더코드, Length: 2891, dtype: object

In [8]:
train_fin = pd.read_pickle("../data/20/train_v2.pkl")

In [10]:
lags_col = ['rolling_mean_7', 'rolling_mean_14', 'rolling_mean_21', 'rolling_mean_28',
            'lag_sales_wd_1', 'lag_sales_wd_2', 'lag_sales_wd_3', 'lag_sales_wd_4', 'lag_sales_wd_5', 
            'lag_sales_wk_1', 'lag_sales_wk_2']
train_fin_wd_lag = train_fin.loc[train_fin.weekends==0]
train_fin_wk_lag = train_fin.loc[train_fin.weekends==1]
train_fin_wd_no_lag = train_fin.loc[train_fin.weekends==0].drop(lags_col,  axis = 1)
train_fin_wk_no_lag = train_fin.loc[train_fin.weekends==1].drop(lags_col,  axis = 1)
# train_fin_light_ver = train_fin

In [11]:
train_fin_wk_lag.to_pickle("../data/20/train_fin_wk_lag.pkl")
train_fin_wd_lag.to_pickle("../data/20/train_fin_wd_lag.pkl")
train_fin_wk_no_lag.to_pickle("../data/20/train_fin_wk_no_lag.pkl")
train_fin_wd_no_lag.to_pickle("../data/20/train_fin_wd_no_lag.pkl")
# train_fin_light_ver.to_pickle("../data/20/train_fin_light_ver.pkl")

In [7]:
test_fin = pd.read_pickle('../data/20/test_v2.pkl')

In [8]:
lags_col = ['rolling_mean_7', 'rolling_mean_14', 'rolling_mean_21', 'rolling_mean_28',
            'lag_sales_wd_1', 'lag_sales_wd_2', 'lag_sales_wd_3', 'lag_sales_wd_4', 'lag_sales_wd_5', 
            'lag_sales_wk_1', 'lag_sales_wk_2']
test_fin_wd_lag = test_fin.loc[test_fin.weekends==0]
test_fin_wk_lag = test_fin.loc[test_fin.weekends==1]
test_fin_wd_no_lag = test_fin.loc[test_fin.weekends==0].drop(lags_col,  axis = 1)
test_fin_wk_no_lag = test_fin.loc[test_fin.weekends==1].drop(lags_col,  axis = 1)
# train_fin_light_ver = train_fin

In [9]:
test_fin_wk_lag.to_pickle("../data/20/test_fin_wk_lag.pkl")
test_fin_wd_lag.to_pickle("../data/20/test_fin_wd_lag.pkl")
test_fin_wk_no_lag.to_pickle("../data/20/test_fin_wk_no_lag.pkl")
test_fin_wd_no_lag.to_pickle("../data/20/test_fin_wd_no_lag.pkl")
# train_fin_light_ver.to_pickle("../data/20/train_fin_light_ver.pkl")

In [10]:
lags_click = ['small_click_r', 'mid_click_r', 'big_click_r', 'age30_small', 'age40_small', 'age50_small',
       'age60above_small', 'age30_middle', 'age40_middle', 'age50_middle',
       'age60above_middle', 'age30_big', 'age40_big', 'age50_big',
       'age60above_big', 'pc_small', 'mobile_small', 'pc_middle',
       'mobile_middle', 'pc_big', 'mobile_big']

test_fin_wk_lag_noclick = test_fin_wk_lag.drop(lags_click, axis=1)
test_fin_wd_lag_noclick = test_fin_wd_lag.drop(lags_click, axis=1)

test_fin_wk_lag_noclick.to_pickle("../data/20/test_fin_wk_lag_noclick.pkl")
test_fin_wd_lag_noclick.to_pickle("../data/20/test_fin_wd_lag_noclick.pkl")

In [ ]:
train_fin_wk_lag_noclick = train_fin_wk_lag.drop(lags_click, axis=1)
train_fin_wd_lag_noclick = train_fin_wd_lag.drop(lags_click, axis=1)

train_fin_wk_lag_noclick.to_pickle("../data/20/train_fin_wk_lag_noclick.pkl")
train_fin_wd_lag_noclick.to_pickle("../data/20/train_fin_wd_lag_noclick.pkl")

In [4]:
import pandas as pd
import datetime

In [6]:
df2 = pd.read_excel("../data/01/2020sales_test_added.xlsx")
df2 = df2.drop(2891, axis=0)
temp2 = pd.merge(left = temp, right=df2[['상품명','original_c']].drop_duplicates(), on='상품명', how='left')
full_train = pd.read_pickle("../data/20/train_fin_light_ver.pkl")
train_dec = full_train.loc[(full_train.months == 12)]
train_dec.sort_values(['방송일시', '상품코드'], ascending=[True, True], inplace=True)
train_dec_lags = train_dec[['original_c', '취급액']].groupby(['original_c']).mean()
train_dec_lags.reset_index(inplace=True)


In [7]:
            

            # extract 2019-Jun data
train_jun = full_train.loc[(full_train.months == 6)]
train_jun.sort_values(['방송일시', '상품코드'], ascending=[True, True], inplace=True)
train_jun_lags = train_jun[['original_c', '취급액']].groupby(['original_c']).mean()
train_jun_lags.reset_index(inplace=True)


In [8]:
train_mso = pd.concat([train_jun_lags, train_dec_lags]).groupby(['original_c']).mean()
train_mso.reset_index(inplace=True)
train_mso = train_mso.rename(columns = {'취급액':'mean_sales_origin'})

In [21]:
train_mso

,original_c,mean_sales_origin
0,18k,7.790800e+06
1,TV,1.868551e+07
2,가구세트,1.236267e+07
3,가마솥,4.726020e+07
4,가스레인지,1.651674e+07
...,...,...
159,홍어,3.378870e+07
160,홍합환,2.295633e+07
161,황태,3.190000e+07
162,휴지,5.095783e+07


In [20]:
len(train_mso.mean_sales_origin.unique())

164

In [47]:
train_mso.original_c.str.encode('utf-8')[1:10]

1                                                b'TV'
2    b'\xea\xb0\x80\xea\xb5\xac\xec\x84\xb8\xed\x8a...
3              b'\xea\xb0\x80\xeb\xa7\x88\xec\x86\xa5'
4    b'\xea\xb0\x80\xec\x8a\xa4\xeb\xa0\x88\xec\x9d...
5    b'\xea\xb0\x81\xec\xa7\x88\xec\xa0\x9c\xea\xb1...
6    b'\xea\xb0\x84\xea\xb3\xa0\xeb\x93\xb1\xec\x96...
7              b'\xea\xb0\x88\xeb\xb9\x84\xed\x83\x95'
8                          b'\xea\xb0\x88\xec\xb9\x98'
9              b'\xea\xb0\x90\xec\x9e\x90\xed\x83\x95'
Name: original_c, dtype: object

In [78]:
def rreplace(s, old, new, occurrence):
    li = s.rsplit(old, occurrence)
    return new.join(li)

In [96]:
train_mso.original_c.str.encode('utf-8')[2]

b'\xea\xb0\x80\xea\xb5\xac\xec\x84\xb8\xed\x8a\xb8'

In [87]:
d = rreplace(train_mso.original_c.str.encode('utf-8')[1:2], '.', '', train_mso.original_c.str.encode('utf-8')[1:2].count('.') - 4) 
print(d)

KeyError: 'Requested level (.) does not match index name (None)'

In [77]:
train_mso.original_c.str.encode('utf-8')[1:20]

1                                                 b'TV'
2     b'\xea\xb0\x80\xea\xb5\xac\xec\x84\xb8\xed\x8a...
3               b'\xea\xb0\x80\xeb\xa7\x88\xec\x86\xa5'
4     b'\xea\xb0\x80\xec\x8a\xa4\xeb\xa0\x88\xec\x9d...
5     b'\xea\xb0\x81\xec\xa7\x88\xec\xa0\x9c\xea\xb1...
6     b'\xea\xb0\x84\xea\xb3\xa0\xeb\x93\xb1\xec\x96...
7               b'\xea\xb0\x88\xeb\xb9\x84\xed\x83\x95'
8                           b'\xea\xb0\x88\xec\xb9\x98'
9               b'\xea\xb0\x90\xec\x9e\x90\xed\x83\x95'
10              b'\xea\xb1\xb4\xec\xa1\xb0\xea\xb8\xb0'
11              b'\xea\xb3\xa0\xea\xb5\xac\xeb\xa7\x88'
12              b'\xea\xb3\xa8\xeb\xb1\x85\xec\x9d\xb4'
13                          b'\xea\xb3\xb0\xed\x83\x95'
14    b'\xea\xb3\xb1\xec\xb0\xbd\xec\xa0\x84\xea\xb3...
15              b'\xea\xb3\xbc\xeb\xa9\x94\xea\xb8\xb0'
16                          b'\xea\xb7\xb8\xeb\xa6\xb4'
17                          b'\xea\xb8\x88\xea\xb3\xa0'
18              b'\xea\xb8\xb0\xeb\x8a\xa5\xec\x

In [75]:
temp2.original_c.str.encode('utf-8').unique()

array([b'\xec\x85\x94\xec\xb8\xa0', b'\xed\x8c\xac\xed\x8b\xb0',
       b'\xed\x97\xa4\xec\x96\xb4\xec\x97\x90\xec\x84\xbc\xec\x8a\xa4',
       b'\xeb\x83\x84\xeb\xb9\x84',
       b'\xec\x9d\xb4\xeb\x84\x88\xec\x9b\xa8\xec\x96\xb4',
       b'\xea\xb9\x80\xec\xb9\x98', b'\xeb\xb8\x8c\xeb\x9d\xbc',
       b'\xeb\xac\xb4\xed\x98\x95',
       b'\xec\x84\xa0\xec\x8a\xa4\xed\x8b\xb1',
       b'\xea\xb0\x88\xec\xb9\x98',
       b'\xeb\xaa\xa8\xec\x8b\x9c\xeb\x96\xa1',
       b'\xec\x84\xb8\xed\x83\x81\xea\xb8\xb0',
       b'\xed\x8b\xb0\xec\x85\x94\xec\xb8\xa0',
       b'\xeb\xb9\x84\xeb\x8d\xb0', b'TV', b'\xec\xb9\xa8\xeb\x8c\x80',
       b'\xec\x83\xa4\xec\x9b\x8c\xea\xb8\xb0',
       b'\xed\x86\xa0\xed\x8a\xb8\xeb\xb0\xb1',
       b'\xeb\xaa\xa8\xec\x9e\x90',
       b'\xeb\xaf\xb9\xec\x84\x9c\xea\xb8\xb0',
       b'\xec\x83\xb4\xed\x91\xb8',
       b'\xeb\xb2\xa0\xec\x8a\xa4\xed\x8a\xb8',
       b'\xeb\x8f\x84\xea\xb0\x80\xeb\x8b\x88\xed\x83\x95',
       b'\xec\x9e\x90\xec\xbc\x93', b'\xeb

In [45]:
for i in temp2.original_c.str.encode('utf-8').unique():
    print( i in train_mso.original_c.str.encode('utf-8'))

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


In [11]:

temp3 = pd.merge(left=temp2, right=train_mso, how='left', on='original_c')

In [22]:
temp3

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,exposed,ymd,original_c,mean_sales_origin
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,20.000000,2020-06-01,셔츠,1.243643e+07
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,20.000000,2020-06-01,셔츠,1.243643e+07
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,20.000000,2020-06-01,셔츠,1.243643e+07
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN,20.000000,2020-06-01,팬티,3.894409e+07
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN,20.000000,2020-06-01,팬티,3.894409e+07
...,...,...,...,...,...,...,...,...,...,...,...,...
2886,2020-07-01 00:20:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,20.000000,2020-07-01,무형,NaN
2887,2020-07-01 00:40:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,20.000000,2020-07-01,무형,NaN
2888,2020-07-01 01:00:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,20.000000,2020-07-01,무형,NaN
2889,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,NaN,20.000000,2020-07-01,레깅스,1.796169e+07


In [4]:
full_train = pd.read_pickle("../data/20/train_fin_light_ver.pkl")

In [96]:
month_ori = full_train[['months','original_c','취급액']].groupby(['months','original_c']).mean()
month_ori.reset_index(inplace=True)
month_ori = month_ori.rename(columns = {'취급액':'ori'})

In [101]:
full_train2 = pd.merge(left = full_train, right = month_ori, on = ['months','original_c'], how = 'left')
full_train2

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,volume,exposed,...,rolling_mean_origin_7,rolling_mean_origin_14,rolling_mean_origin_21,rolling_mean_origin_28,lag_sales_1,lag_sales_2,lag_sales_5,lag_sales_7,ts_pred,ori
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900.0,2099000.0,52.606516,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.434417e+07,1.944518e+07
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900.0,4371000.0,109.548872,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.434417e+07,1.944518e+07
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900.0,3262000.0,81.754386,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.434417e+07,1.944518e+07
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900.0,6955000.0,174.310777,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.434417e+07,1.944518e+07
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900.0,6672000.0,167.218045,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.434417e+07,1.944518e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35374,2019-12-31 23:40:00,NaN,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000.0,10157000.0,68.628378,20.0,...,1.624279e+07,1.612416e+07,1.594011e+07,1.639085e+07,149122000.0,263222000.0,178271000.0,79018000.0,1.506757e+08,1.565429e+07
35375,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000.0,50929000.0,286.117978,20.0,...,1.507896e+07,1.480849e+07,1.531213e+07,1.583262e+07,108945000.0,144830000.0,120211000.0,55727000.0,1.014801e+08,1.221779e+07
35376,2020-01-01 00:00:00,NaN,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000.0,13765000.0,87.120253,20.0,...,1.507896e+07,1.480849e+07,1.531213e+07,1.583262e+07,108945000.0,144830000.0,120211000.0,55727000.0,1.014801e+08,1.221779e+07
35377,2020-01-01 00:00:00,NaN,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000.0,104392000.0,621.380952,20.0,...,1.507896e+07,1.480849e+07,1.531213e+07,1.583262e+07,108945000.0,144830000.0,120211000.0,55727000.0,1.014801e+08,1.221779e+07


In [99]:
df2 = pd.read_excel("../data/01/2020sales_test_added.xlsx")
df2 = df2.drop(2891, axis=0)

temp2 = pd.merge(left = temp, right=df2[['상품명','original_c']], on='상품명', how='left')


In [100]:
temp2

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,exposed,ymd,original_c
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,20.000000,2020-06-01,셔츠
1,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,20.000000,2020-06-01,셔츠
2,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,20.000000,2020-06-01,셔츠
3,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,20.000000,2020-06-01,셔츠
4,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,20.000000,2020-06-01,셔츠
...,...,...,...,...,...,...,...,...,...,...,...
40104,2020-07-01 01:00:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,20.000000,2020-07-01,무형
40105,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,NaN,20.000000,2020-07-01,레깅스
40106,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,NaN,20.000000,2020-07-01,레깅스
40107,2020-07-01 01:40:00,15.933333,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,NaN,15.933333,2020-07-01,레깅스


In [11]:

full_train = pd.read_pickle("../data/20/train_fin_light_ver.pkl")

train_dec = full_train.loc[(full_train.months == 12)]
train_dec.sort_values(['방송일시', '상품코드'], ascending=[True, True], inplace=True)
lag_cols = ['original_c', 'rolling_mean_7', 'rolling_mean_14',\
            'rolling_mean_21','rolling_mean_28']
train_dec_lags = train_dec[lag_cols].groupby(['original_c']).mean()
train_dec_lags.reset_index(inplace=True)

train_jun = full_train.loc[(full_train.months == 6)]
train_jun.sort_values(['방송일시', '상품코드'], ascending=[True, True], inplace=True)
train_jun_lags = train_jun[lag_cols].groupby(['original_c']).mean()
train_jun_lags.reset_index(inplace=True)

train_lags = pd.concat([train_jun_lags, train_dec_lags]).groupby(['original_c']).mean()
#train_lags = train_lags[lag_cols].groupby(['days', 'original_c']).mean()
train_lags.reset_index(inplace=True)

temp = pd.merge(left=temp, right=train_lags, how='left', on=['original_c'])

In [12]:
temp

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,exposed,ymd,original_c,rolling_mean_7,rolling_mean_14,rolling_mean_21,rolling_mean_28
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,20.000000,2020-06-01,셔츠,2.429492e+07,2.450378e+07,2.484560e+07,2.500155e+07
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,20.000000,2020-06-01,셔츠,2.429492e+07,2.450378e+07,2.484560e+07,2.500155e+07
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,20.000000,2020-06-01,셔츠,2.429492e+07,2.450378e+07,2.484560e+07,2.500155e+07
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN,20.000000,2020-06-01,팬티,2.919025e+07,2.944196e+07,2.945222e+07,2.902265e+07
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN,20.000000,2020-06-01,팬티,2.919025e+07,2.944196e+07,2.945222e+07,2.902265e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2886,2020-07-01 00:20:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,20.000000,2020-07-01,무형,NaN,NaN,NaN,NaN
2887,2020-07-01 00:40:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,20.000000,2020-07-01,무형,NaN,NaN,NaN,NaN
2888,2020-07-01 01:00:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,20.000000,2020-07-01,무형,NaN,NaN,NaN,NaN
2889,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,NaN,20.000000,2020-07-01,레깅스,2.304329e+07,2.433598e+07,2.485452e+07,2.537607e+07


In [60]:
df = pd.read_excel("../data/00/202006schedule.xlsx", skiprows=1)

In [61]:
df['exposed'] = df['노출(분)']
df.마더코드 = df.마더코드.astype(int).astype(str).str.zfill(6)
df.상품코드 = df.상품코드.astype(int).astype(str).str.zfill(6)
df.방송일시 = pd.to_datetime(df.방송일시, format="%Y/%m/%d %H:%M")
df.sort_values(['방송일시', '상품코드'], ascending=[True, True], inplace=True)
df['ymd'] = [d.date() for d in df["방송일시"]]
temp = df

# define ts_schedule, one row for each timeslot

# then 방송일시 works as index in ts_schedule

# temp['ts_schedule'] = df.copy().groupby('방송일시').first()

# temp['ts_schedule'].reset_index(inplace=True)

In [63]:
df2 = pd.read_excel("../data/01/2020sales_test_added.xlsx")
df2 = df2.drop(2891, axis=0)

In [79]:
temp = pd.merge(left = temp, right=df2[['상품명','middle_c']].drop_duplicates(), on='상품명', how='left')
temp['days'] = temp.방송일시.dt.day

In [5]:
full_train = pd.read_pickle("../data/20/train_fin_light_ver.pkl")

In [81]:
train_dec = full_train.loc[(full_train.months == 12)]
train_dec.sort_values(['방송일시', '상품코드'], ascending=[True, True], inplace=True)
train_dec['days'] = train_dec.days - 1
lag_cols = ['days', 'middle_c', 'rolling_mean_origin_7', 'rolling_mean_origin_14',\
            'rolling_mean_origin_21','rolling_mean_origin_28']
train_dec_lags = train_dec[lag_cols].groupby(['days', 'middle_c']).mean()
train_dec_lags.reset_index(inplace=True)

c:\users\baoro\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\baoro\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [82]:
train_jun = full_train.loc[(full_train.months == 6) | ((full_train.months == 7) & (full_train.ymd < datetime.date(2019, 7, 3)))]
# train_jun = full_train.loc[(full_train.months == 7) & (full_train.ymd < datetime.date(2019, 7, 3))]
train_jun.days.loc[train_jun.방송일시.dt.month == 7] = train_jun.loc[train_jun.방송일시.dt.month == 7]['days'] + 30

# 7월 days에 30 더하기
train_jun.sort_values(['방송일시', '상품코드'], ascending=[True, True], inplace=True)
train_jun['days'] = train_jun.days - 2
lag_cols = ['days', 'middle_c', 'rolling_mean_origin_7', 'rolling_mean_origin_14',\
            'rolling_mean_origin_21','rolling_mean_origin_28']
train_jun_lags = train_jun[lag_cols].groupby(['days', 'middle_c']).mean()
train_jun_lags.reset_index(inplace=True)

c:\users\baoro\anaconda3\envs\tensorflow\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
c:\users\baoro\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\baoro\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org

In [84]:
train_lags = pd.concat([train_jun_lags, train_dec_lags]).groupby(['days','middle_c']).mean()
#train_lags = train_lags[lag_cols].groupby(['days', 'original_c']).mean()


In [85]:
train_lags.reset_index(inplace=True)

In [89]:
train_lags.loc[train_lags.middle_c == "거실가구"]

,days,middle_c,rolling_mean_origin_7,rolling_mean_origin_14,rolling_mean_origin_21,rolling_mean_origin_28
0,-1,거실가구,NaN,7.440692e+06,7.966590e+06,7.965464e+06
70,3,거실가구,1.009107e+07,1.009107e+07,8.556722e+06,8.414535e+06
113,5,거실가구,8.569241e+06,8.569241e+06,8.219929e+06,8.171459e+06
156,7,거실가구,NaN,NaN,NaN,9.980613e+06
176,8,거실가구,9.852333e+06,9.852333e+06,9.852333e+06,9.969294e+06
199,9,거실가구,8.250484e+06,8.850674e+06,8.850674e+06,8.445035e+06
225,10,거실가구,2.270000e+07,2.270000e+07,2.270000e+07,2.270000e+07
268,12,거실가구,9.082455e+06,8.842403e+06,8.842403e+06,8.599440e+06
380,17,거실가구,7.283500e+06,8.207881e+06,8.589608e+06,8.589608e+06
572,26,거실가구,NaN,NaN,1.797944e+07,1.797944e+07


In [87]:
temp = pd.merge(left=temp, right=train_lags, how='left', on=['days', 'middle_c'])

In [53]:
def check_na(df):
    """
    :objective: show na
    :return: columns with na / na counts
    """
    print(df.isnull().sum())

In [88]:
check_na(temp)

방송일시                           0
노출(분)                       1135
마더코드                           0
상품코드                           0
상품명                            0
상품군                            0
판매단가                           0
취급액                         2921
exposed                     1135
ymd                            0
original_c                    42
days                           0
rolling_mean_origin_7_x     1945
rolling_mean_origin_14_x    1913
rolling_mean_origin_21_x    1904
rolling_mean_origin_28_x    1895
middle_c                     178
rolling_mean_origin_7_y     1256
rolling_mean_origin_14_y    1203
rolling_mean_origin_21_y    1182
rolling_mean_origin_28_y    1176
dtype: int64


In [11]:
import pandas as pd
import datetime

In [2]:
tmp = pd.read_pickle('../data/20/train_fin_light_ver.pkl')
tmp.columns

Index(['방송일시', '노출(분)', '마더코드', '상품코드', '상품명', '상품군', '판매단가', '취급액', 'volume',
       'exposed', 'ymd', 'years', 'months', 'days', 'hours', 'week_num',
       'weekdays', 'hours_inweek', 'holidays', 'red', 'weekends', 'min_start',
       'japp', 'parttime', 'min_range', 'show_id', 'small_c', 'small_c_code',
       'middle_c', 'middle_c_code', 'big_c', 'big_c_code', 'original_c',
       'primetime', 'prime_smallc', 'sales_power', 'freq', 'dup_times',
       'dup_times_smallc', 'lag_scode_count', 'lag_mcode_price',
       'lag_mcode_count', 'lag_bigcat_price', 'lag_bigcat_count',
       'lag_bigcat_price_day', 'lag_bigcat_count_day', 'lag_small_c_price',
       'lag_small_c_count', 'lag_all_price_show', 'lag_all_price_day',
       'bpower', 'steady', 'men', 'luxury', 'pay', 'spring', 'summer', 'fall',
       'winter', 'small_click_r', 'mid_click_r', 'big_click_r', 'rain',
       'temp_diff_s', 'exposed_t', 'age30_small', 'age40_small', 'age50_small',
       'age60above_small', 'age30_mid

In [3]:
tmp = pd.read_pickle('../data/20/train_fin_wd_lag.pkl')
tmp.columns

Index(['방송일시', '노출(분)', '마더코드', '상품코드', '상품명', '상품군', '판매단가', '취급액', 'volume',
       'exposed', 'ymd', 'years', 'months', 'days', 'hours', 'week_num',
       'weekdays', 'hours_inweek', 'holidays', 'red', 'weekends', 'min_start',
       'japp', 'parttime', 'min_range', 'show_id', 'small_c', 'small_c_code',
       'middle_c', 'middle_c_code', 'big_c', 'big_c_code', 'original_c',
       'primetime', 'prime_smallc', 'sales_power', 'freq', 'dup_times',
       'dup_times_smallc', 'lag_scode_count', 'lag_mcode_price',
       'lag_mcode_count', 'lag_bigcat_price', 'lag_bigcat_count',
       'lag_bigcat_price_day', 'lag_bigcat_count_day', 'lag_small_c_price',
       'lag_small_c_count', 'lag_all_price_show', 'lag_all_price_day',
       'bpower', 'steady', 'men', 'luxury', 'pay', 'spring', 'summer', 'fall',
       'winter', 'small_click_r', 'mid_click_r', 'big_click_r', 'rain',
       'temp_diff_s', 'exposed_t', 'age30_small', 'age40_small', 'age50_small',
       'age60above_small', 'age30_mid

In [59]:
from features_yj import Features

finish getting all lag data
(2716, 48) : df shape
finish getting brand power data
(2716, 49) : df shape
finish getting season_items data
(2716, 57) : df shape
finish getting click ratio data
(2716, 58) : df shape
finish getting click ratio data
(2716, 59) : df shape
finish getting click ratio data
(2716, 60) : df shape
finish getting weather data
(2716, 62) : df shape
finish getting add_device_click_ratio data
(2716, 81) : df shape
finish getting get_rolling_means data
(2716, 83) : df shape


KeyError: 'original_c'

In [7]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import math
import random

# data
import datetime
import itertools
import json
import pickle

# sklearn
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# custom class
# from features_yj import Features

In [57]:
df = pd.read_excel("../data/00/202006schedule.xlsx", skiprows=1)
df2 = pd.read_excel("../data/01/2020sales_test_added.xlsx")
df2 = df2.drop(2891, axis=0)
df = pd.merge(left = df, right=df2[['상품명','original_c']], on='상품명', how='left')

In [58]:
df

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,original_c
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,셔츠
1,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,셔츠
2,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,셔츠
3,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,셔츠
4,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN,셔츠
...,...,...,...,...,...,...,...,...,...
40104,2020-07-01 01:00:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN,무형
40105,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,NaN,레깅스
40106,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,NaN,레깅스
40107,2020-07-01 01:40:00,15.933333,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,NaN,레깅스


In [5]:
# full_train = pd.read_excel("../data/01/2019sales_added.xlsx")
# full_train.ymd = pd.to_datetime(full_train.ymd).dt.date
full_train2 = pd.read_pickle("../data/20/train_fin_light_ver.pkl")
full_train = pd.merge(left = full_train2,
                      right = full_train[['상품명','original_c']],
                      on = '상품명', how = 'left')


In [13]:
full_train = pd.read_pickle("../data/20/train_fin_light_ver.pkl")
full_train.columns

Index(['방송일시', '노출(분)', '마더코드', '상품코드', '상품명', '상품군', '판매단가', '취급액', 'volume',
       'exposed', 'ymd', 'years', 'months', 'days', 'hours', 'week_num',
       'weekdays', 'hours_inweek', 'holidays', 'red', 'weekends', 'min_start',
       'japp', 'parttime', 'min_range', 'show_id', 'small_c', 'small_c_code',
       'middle_c', 'middle_c_code', 'big_c', 'big_c_code', 'original_c',
       'primetime', 'prime_smallc', 'sales_power', 'freq', 'dup_times',
       'dup_times_smallc', 'lag_scode_count', 'lag_mcode_price',
       'lag_mcode_count', 'lag_bigcat_price', 'lag_bigcat_count',
       'lag_bigcat_price_day', 'lag_bigcat_count_day', 'lag_small_c_price',
       'lag_small_c_count', 'lag_all_price_show', 'lag_all_price_day',
       'bpower', 'steady', 'men', 'luxury', 'pay', 'spring', 'summer', 'fall',
       'winter', 'small_click_r', 'mid_click_r', 'big_click_r', 'rain',
       'temp_diff_s', 'exposed_t', 'age30_small', 'age40_small', 'age50_small',
       'age60above_small', 'age30_mid

In [14]:
train_dec = full_train.loc[(full_train.ymd > datetime.date(2019, 12, 15)) & (full_train.months == 12)]

train_dec.sort_values(['방송일시', '상품코드'], ascending=[True, True], inplace=True)

train_dec['days'] = train_dec.days - 1

lag_cols = ['days', 'original_c', 'rolling_mean_origin_7', 'rolling_mean_origin_14', 'rolling_mean_origin_21', 'rolling_mean_origin_28']


In [15]:
train_dec.columns

Index(['방송일시', '노출(분)', '마더코드', '상품코드', '상품명', '상품군', '판매단가', '취급액', 'volume',
       'exposed', 'ymd', 'years', 'months', 'days', 'hours', 'week_num',
       'weekdays', 'hours_inweek', 'holidays', 'red', 'weekends', 'min_start',
       'japp', 'parttime', 'min_range', 'show_id', 'small_c', 'small_c_code',
       'middle_c', 'middle_c_code', 'big_c', 'big_c_code', 'original_c',
       'primetime', 'prime_smallc', 'sales_power', 'freq', 'dup_times',
       'dup_times_smallc', 'lag_scode_count', 'lag_mcode_price',
       'lag_mcode_count', 'lag_bigcat_price', 'lag_bigcat_count',
       'lag_bigcat_price_day', 'lag_bigcat_count_day', 'lag_small_c_price',
       'lag_small_c_count', 'lag_all_price_show', 'lag_all_price_day',
       'bpower', 'steady', 'men', 'luxury', 'pay', 'spring', 'summer', 'fall',
       'winter', 'small_click_r', 'mid_click_r', 'big_click_r', 'rain',
       'temp_diff_s', 'exposed_t', 'age30_small', 'age40_small', 'age50_small',
       'age60above_small', 'age30_mid

In [16]:
train_dec_lags = train_dec[lag_cols].groupby(['days', 'original_c']).mean()

In [53]:
train_dec_lags.reset_index(inplace=True)

test = pd.merge(left=test, right=train_dec_lags[['days','original_c','rolling_mean_origin_7','rolling_mean_origin_14', 'rolling_mean_origin_21', 'rolling_mean_origin_28']], how='left',
                      on=['days', 'original_c'])

In [54]:
test.columns

Index(['방송일시', '노출(분)', '마더코드', '상품코드', '상품명', '상품군', '판매단가', '취급액',
       'original_c', 'exposed', 'ymd', 'days', 'rolling_mean_origin_7_x',
       'rolling_mean_origin_14_x', 'rolling_mean_origin_7_y',
       'rolling_mean_origin_14_y', 'rolling_mean_origin_21',
       'rolling_mean_origin_28'],
      dtype='object')

In [45]:
df = pd.read_excel("../data/00/202006schedule.xlsx", skiprows=1)
df2 = pd.read_excel("../data/01/2020sales_test_added.xlsx")
df2 = df2.drop(2891, axis=0)

df = pd.merge(left = df, right=df2[['상품명','original_c']], on='상품명', how='left')

In [81]:
tmp1 = pd.DataFrame(df.original_c.unique())
tmp1 = tmp1.rename(columns = {0:'c'})

In [78]:
tmp2 = pd.DataFrame(full_train.original_c.unique())
tmp2 = tmp2.rename(columns = {0:'c'})

In [84]:
tmp1 = df.original_c.unique()
tmp2 = full_train.original_c.unique()

In [87]:
for i in tmp1:
    (i in tmp2)

True
True
True
True
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
True
True
True
True
True
True
True


In [47]:
df['exposed'] = df['노출(분)']
df.마더코드 = df.마더코드.astype(int).astype(str).str.zfill(6)

df.상품코드 = df.상품코드.astype(int).astype(str).str.zfill(6)

df.방송일시 = pd.to_datetime(df.방송일시, format="%Y/%m/%d %H:%M")

df.sort_values(['방송일시', '상품코드'], ascending=[True, True], inplace=True)




df['ymd'] = [d.date() for d in df["방송일시"]]

test = df



In [49]:
test['days'] = test.방송일시.dt.day